# Do Region-based saliency explanations distinguish unaffected and syndromic images?

In [1]:
import math
import numpy as np
import pandas as pd
import pingouin as pg
import seaborn as sns
from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
matplotlib.rc('text', usetex=True)
# matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
plt.rcParams['text.latex.preamble'] = r"\usepackage{bm} \usepackage{amsmath}"

In [2]:
xai_methods = ['Gradient', 'SmoothGrad', 'IntegratedGradients', 'GuidedBackprop', 'ExcitationBackprop', 'DeconvNet',
               'LRP-EpsilonPlus', 'LRP-EpsilonPlusFlat', 'LRP-EpsilonAlpha2Beta1', 'LRP-EpsilonAlpha2Beta1Flat',
               'DeepLIFT', 'GuidedGradCam', 'LayerDeepLIFT', 'Occlusion'] 

is_significant = pd.DataFrame(0, columns=['22q11DS', 'Angelman', 'KS', 'NS', 'WS'], index=['eye', 'nose', 'mouth'])

for method_idx, method in enumerate(xai_methods):
    
    df = pd.read_csv('../results/results-rq1/%s.csv'%method)
    results = pd.DataFrame(columns=['method', 'group1','group1_size', 'group1_mean', 'group1_std', 'group1_normal', 'region', 'group2', 'group2_size', 'group2_mean', 'group2_std', 'group2_normal', 'equal_var', 'cohen_d','test','statistic', 'p_value', 'is_significant'])
    n = 0
    for syndrome in ['22q11DS', 'Angelman', 'KS', 'NS', 'WS'] :
        for region in ['eye','nose','mouth']:

            group1 = df[df['label']=='Unaffected']['%s_xai'%region].to_numpy()
            group2 = df[df['label']==syndrome]['%s_xai'%region].to_numpy()

            cohen_d = pg.compute_effsize(group1, group2, eftype='cohen')
            equal_var = int(pg.homoscedasticity({'group1':group1, 'group2':group2}, method="levene", alpha=.05)['equal_var'])

            group1_normal = int(pg.normality(group1, method="shapiro", alpha=.05)['normal'])
            group2_normal = int(pg.normality(group2, method="shapiro", alpha=.05)['normal'])

            if equal_var==1 & group1_normal==1 & group2_normal==1:
                test_type = 't-test'
                statistic = float(pg.ttest(group1, group2, paired=False, alternative='two-sided')['T'])
                p_value   = float(pg.ttest(group1, group2, paired=False, alternative='two-sided')['p-val'])
            else:
                test_type = 'mwu'
                statistic = float(pg.mwu(group1, group2, alternative='two-sided')['U-val'])
                p_value   = float(pg.mwu(group1, group2, alternative='two-sided')['p-val'])

            results.loc[n] = [method] + ['Unaffected'] + [len(group1), group1.mean(), group1.std(), group1_normal] + [region] + [syndrome] + [len(group2), group2.mean(), group2.std(), group2_normal] +  [equal_var, cohen_d] + [test_type] + [int(statistic), p_value, p_value<0.001]
            n += 1

    # Apply Bonferni correction:
    num_significant_tests = np.sum(results['is_significant'])
    results['p_value_corrected'] = pg.multicomp(list(results['p_value']), alpha=0.001, method='bonf')[1]
    results['is_significant'] = pg.multicomp(list(results['p_value']), alpha=0.001, method='bonf')[0]
    print('%25s \t # of significant tests=%d (%d)'%(method, num_significant_tests, np.sum(results['is_significant'])))
        
    #display(results)
    
    significant_tests_per_method = 0
    for region in ['eye','nose','mouth']:
        
        val1 = int(results[(results['group2']=='22q11DS')   & (results['region']==region)]['is_significant'])
        val2 = int(results[(results['group2']=='Angelman') & (results['region']==region)]['is_significant'])
        val3 = int(results[(results['group2']=='KS')             & (results['region']==region)]['is_significant'])
        val4 = int(results[(results['group2']=='NS')             & (results['region']==region)]['is_significant'])
        val5 = int(results[(results['group2']=='WS')             & (results['region']==region)]['is_significant'])
        
        is_significant.loc[region,'22q11DS'] += val1
        is_significant.loc[region,'Angelman'] += val2
        is_significant.loc[region,'KS'] += val2
        is_significant.loc[region,'NS'] += val4
        is_significant.loc[region,'WS'] += val5
        significant_tests_per_method += (val1+val2+val3+val4+val5)
        
display(is_significant)   
#is_significant.style.to_latex()

                 Gradient 	 # of significant tests=9 (8)
               SmoothGrad 	 # of significant tests=9 (9)
      IntegratedGradients 	 # of significant tests=9 (9)
           GuidedBackprop 	 # of significant tests=12 (12)
       ExcitationBackprop 	 # of significant tests=10 (9)
                DeconvNet 	 # of significant tests=6 (5)
          LRP-EpsilonPlus 	 # of significant tests=5 (4)
      LRP-EpsilonPlusFlat 	 # of significant tests=9 (6)
   LRP-EpsilonAlpha2Beta1 	 # of significant tests=8 (8)
LRP-EpsilonAlpha2Beta1Flat 	 # of significant tests=8 (8)
                 DeepLIFT 	 # of significant tests=10 (6)
            GuidedGradCam 	 # of significant tests=13 (12)
            LayerDeepLIFT 	 # of significant tests=11 (9)
                Occlusion 	 # of significant tests=7 (6)


,22q11DS,Angelman,KS,NS,WS
eye,11,2,2,9,9
nose,9,4,4,4,9
mouth,3,8,8,6,11
